In [13]:
# imports
import sys
sys.path.append( '../FIA' )
sys.path.append( '../ML' )

from FIA import *
from ML4com import *

# TensorFlow and tf.keras
import tensorflow as tf
import keras
from keras import layers, activations


# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [2]:
info_dir = "../../data/comm8_self"
data_dir = "../../runs/FIA/comm8/oms"
run_dir = "../../runs/ML/try"

info_dir = os.path.normpath(os.path.join(os.getcwd(), info_dir))
data_dir = os.path.normpath(os.path.join(os.getcwd(), data_dir))
run_dir = os.path.normpath(os.path.join(os.getcwd(), run_dir))

strains = pd.read_csv(os.path.join(info_dir, "strains.tsv"), sep="\t")
comm8 = pd.read_csv(os.path.join(info_dir, "comm8.tsv"), sep="\t")

fia_df = load_fia_df(data_dir, file_ending=".mzML", separator="\t")

Loading experiments:


100%|██████████| 68/68 [00:01<00:00, 64.25it/s]


Loading names:


100%|██████████| 72/72 [00:00<00:00, 71988.05it/s]


In [3]:
# binned_dfs = bin_df_stepwise_batch(fia_df, binning_var="mz", binned_var="inty", statistic="sum", start=50.0, stop=1700.0, step=0.002)
# binned_dfs.to_csv(os.path.join(run_dir, "data_matrix.tsv"), sep="\t")
binned_dfs = pd.read_csv(os.path.join(run_dir, "data_matrix.tsv"), sep="\t", index_col="mz", engine="pyarrow")

In [4]:
scaler = MaxAbsScaler()
binned_dfs[:] =  scaler.fit_transform(binned_dfs)

In [5]:
print(binned_dfs.shape)
print(comm8.shape)
print(strains.shape)

(825000, 68)
(68, 8)
(8, 1)


## Model definition

In [20]:
model = keras.Sequential(name="MS_community_prediction")
model.add(keras.Input(shape=(8,68,825000)))
model.add(layers.Dense(1000, activation=activations.tanh, name="Feature_combination_1"))
model.add(layers.Dense(100, activation=activations.relu, name="Feature_combination_2"))
model.add(layers.Dense(100,  activation=activations.relu, name="Feature_interpretation_1"))
model.add(layers.Dense(100,  activation=activations.relu, name="Feature_interpretation_2"))
model.add(layers.Dense(100,  activation=activations.sigmoid, name="Feature_separation_1"))
model.add(layers.Dense(8,  activation=activations.softmax, name="Decision"))

model.summary()


Model: "MS_community_prediction"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Feature_combination_1 (Den  (None, 8, 68, 1000)       825001000 
 se)                                                             
                                                                 
 Feature_combination_2 (Den  (None, 8, 68, 100)        100100    
 se)                                                             
                                                                 
 Feature_interpretation_1 (  (None, 8, 68, 100)        10100     
 Dense)                                                          
                                                                 
 Feature_interpretation_2 (  (None, 8, 68, 100)        10100     
 Dense)                                                          
                                                                 
 Feature_separation_1 (Dens  (None, 8, 68, 

## Splitting dataset

In [35]:
binned_dfs.shape[1]

68

In [ ]:
X = binned_dfs
y = comm8

test_index = sample_without_replacement(68, math.ceil(68/5))
test_data  = X.iloc[test_index]
test_labels = y.iloc[test_index]

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)        # kf = KFold(n_splits = 5)
for train_index, val_index in skf.split(X, y):
	training_data = X.iloc[train_index]
	training_labels = y.iloc[train_index]
	validation_data = X.iloc[val_index]
	validation_labels = y.iloc[val_index]

	model.fit(training_data, training_labels, epochs=10)
	test_loss, test_acc = model.evaluate(validation_data,  validation_labels, verbose="auto")
	
	probability_model = keras.Sequential([model, keras.layers.Softmax()])
	predictions = probability_model.predict(test_data)

	

In [36]:
from sklearn.utils.random import sample_without_replacement

array([61, 55, 67, 62, 66, 27, 51, 21,  0, 50, 17, 36, 12, 59])